<a href="https://colab.research.google.com/github/randreove/TS/blob/exercises/notebooks/60_Forecasting%20II%20Deep%20Learning/2_Early%20Stopping/A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/randreove/TS.git

Cloning into 'TS'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (367/367), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 367 (delta 101), reused 320 (delta 62), pack-reused 0 (from 0)
Receiving objects: 100% (367/367), 27.18 MiB | 11.65 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Filtering content: 100% (17/17), 142.02 MiB | 46.81 MiB/s, done.


# Data Preprocessing for LSTM

## Load Dataset

In [2]:
import pandas as pd

df = pd.read_csv('/content/TS/notebooks/60_Forecasting II Deep Learning/data/precio_mercado_diario.csv', parse_dates=['datetime_utc'], index_col=0)
df = df.loc["2020":].copy()
df

,Mercado SPOT
datetime_utc,
2020-01-01 00:00:00+00:00,38.60
2020-01-01 01:00:00+00:00,36.55
2020-01-01 02:00:00+00:00,32.32
2020-01-01 03:00:00+00:00,30.85
2020-01-01 04:00:00+00:00,30.14
...,...
2023-08-04 17:00:00+00:00,70.00
2023-08-04 18:00:00+00:00,86.30
2023-08-04 19:00:00+00:00,94.00


## Data Preprocessing

### Data Numpy Array

In [3]:
data_array = df.values

### Train Test Split

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_array, test_size=0.2, shuffle=False)

### Create Sequences

In [9]:
import numpy as np

def create_sequences(data, seq_len):
  Xs, ys = [], []

  for i in range(len(data) - seq_len):
    X = data[i:seq_len+i]
    Xs.append(X)

    y = data[seq_len+i]
    ys.append(y)

  Xs = np.array(Xs)
  ys = np.array(ys)

  return Xs, ys

X_test, y_test = create_sequences(test, seq_len=24)
X_train, y_train = create_sequences(train, seq_len=24)


In [10]:
X_train.shape

(25222, 24, 1)

In [11]:
y_train[0]

array([31.98])

## Modelling

### Design NN Architecture

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

model = Sequential(layers=[
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(units=50),
    Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

### Train Model with Early Stopping

In [13]:
from keras.callbacks import EarlyStopping

In [14]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [15]:
model.fit(
    X_train, y_train, validation_data=(X_test, y_test),
    epochs=50, verbose=1, callbacks=[early_stop]
)

Epoch 1/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 15443.9590 - val_loss: 4616.6572
Epoch 2/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 9810.8398 - val_loss: 2111.3899
Epoch 3/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 6975.1665 - val_loss: 873.5867
Epoch 4/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 4617.0024 - val_loss: 382.8892
Epoch 5/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 3188.6301 - val_loss: 207.9234
Epoch 6/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 2104.6245 - val_loss: 163.0330
Epoch 7/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 1433.6365 - val_loss: 140.4496
Epoch 8/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 1194.3558 - val_loss: 134.0660
Epoch 9/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 799.3307 - val_loss: 132.5888
Epoch 10/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 610.8731 - val_loss: 131.5223
Epoch 11/50
789/789 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 549.1431 - val_loss: 138.67

### Calculate Predictions

In [16]:
model.predict(x=X_test)

197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[122.46165 ],
       [103.23145 ],
       [ 92.87982 ],
       ...,
       [ 92.558334],
       [ 92.91867 ],
       [ 89.9477  ]], dtype=float32)

In [17]:
y_test

array([[105.  ],
       [ 90.52],
       [ 87.  ],
       ...,
       [ 94.  ],
       [ 94.  ],
       [ 91.7 ]])